In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

2025-07-28 11:55:56.740658: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-28 11:55:56.749930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753683956.761495   85593 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753683956.765029   85593 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753683956.774039   85593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Path to your dataset
base_dir = "/home/jay/Face_Analysis/datasets/skin_diseases"

# Automatically get sorted class names (folder names)
class_labels = sorted(os.listdir(base_dir))
images_per_class = 2
num_classes = len(class_labels)

# grid plot
plt.figure(figsize=(images_per_class * 3, num_classes * 2.5))

for class_idx, class_name in enumerate(class_labels):
    class_dir = os.path.join(base_dir, class_name)
    image_files = os.listdir(class_dir)[:images_per_class] 

    for img_idx, img_file in enumerate(image_files):
        img_path = os.path.join(class_dir, img_file)
        img = Image.open(img_path)

        plot_index = class_idx * images_per_class + img_idx + 1
        plt.subplot(num_classes, images_per_class, plot_index)
        plt.imshow(img)
        plt.axis("off")
        plt.title(class_name, fontsize=10)

plt.suptitle(" Skin Disease Dataset — 2 Samples per Class", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()
